<div style="line-height:0.5">
<h1 style="color:#DE46D3"> Multiprocessing in Python </h1>
<h4> Processes, Events, and Queues. </h4>
<div style="margin-top: -10px;">
<span style="display: inline-block;">
    <h3 style="color: lightblue; display: inline;">Keywords:</h3> threading + time.sleep
</span>
</div>

In [9]:
import os
import sys
import time
import random

import multiprocessing
from multiprocessing import Process, Pool, Pipe, Event, Lock, Value, current_process, Manager
from multiprocessing.queues import Queue
from multiprocessing.managers import BaseManager

from queue import LifoQueue

import threading

<h2 style="color:#DE46D3"> Example 1 </h2>
wrong usage

In [ ]:
%%script echo skipping 

def take_0(queue0, queue1, queue2, queue3, event0, event1, event2, event3):
    """ Retrieve the message from queue0, if it is not empty, and if the event0 is cleared.\\
    Continuously monitor the queue0 and event0
    N.B.
    works "in reverse!" These methods check if the event is clear to go on...not when it is set
    """
    while True:
        if not event0.is_set():
            if not queue0.empty():
                message = queue0.get()
                if message==1:
                    event1.clear()
                    event2.clear()
                    queue1.put(0)
                    queue2.put(0)
                    event0.set()

def take_1(queue1, event1, event3, queue3):
    """ Retrieve the message from the queue1, if queue2 not empty, when event1 is cleared.\\
    Continuously monitor the queue1 and event1
    """
    while True:
        if not event1.is_set():
            if not queue1.empty():
                message = queue1.get()
                ran = random.randint(0,10)
                if ran in [0,1,2]:
                    print("run found take1")
                    event3.clear()
                    event1.set()
                    time.sleep(2)
                    queue3.put("STOP")
                else:
                    print("start take 1 execution")
                    if message == "STOP":
                        event1.set()

def take_2(queue2, event2, event3, queue3):
    """ Retrieve the message from the queue2, if queue2 not empty, when event2 is cleared.\\
    Continuously monitor the queue2 and event2.
    """
    while True:
        if not event2.is_set():
            if not queue2.empty():
                message = queue2.get()
                ran = random.randint(0,10)
                if ran in [0,1,2]:
                    print("run found take2")
                    event3.clear()
                    queue3.put(3)
                    # Pause for 2 seconds
                    time.sleep(2)
                    event2.set()
                else:
                    print("start take 2 execution")
                    event2.set()

def take_3(queue3, event3, event0):
    """ Retrieve the message from the queue3, if queue3 not empty, when event3 is cleared.\\
    Continuously monitor the queue3 and event3.
    """
    while True:
        if not event3.is_set():
            if not queue3.empty():
                message = queue3.get()
                if message == "STOP":
                    event3.set()
                    event0.clear()
                else:
                    print("take 3:", message)


In [ ]:
%%script echo Skipping
""" Not to run as it is here! 
This snippet is left as example to continue forever. To interrupt in other ways """ 
queue0 = multiprocessing.Queue()
queue1 = multiprocessing.Queue()
queue2 = multiprocessing.Queue()
queue3 = multiprocessing.Queue()

event0 = multiprocessing.Event()
event1 = multiprocessing.Event()
event2 = multiprocessing.Event()
event3 = multiprocessing.Event()

process0 = multiprocessing.Process(target=take_0, args=(queue0,queue1,queue2,queue3,event0,event1,event2,event3))
process1 = multiprocessing.Process(target=take_1, args=(queue1,event1,event3, queue3))
process2 = multiprocessing.Process(target=take_2, args=(queue2,event2,event3, queue3))
process3 = multiprocessing.Process(target=take_3, args=(queue3,event3,event0))

process0.start()
process1.start()
process2.start()
process3.start()

queue0.put(1)
event0.clear()

#for i in range(10):
#    queue1.put(i)
#    queue1.put("STOP")

process0.join()
process1.join()
process2.join()
process3.join()

process0.shutdown()
process1.shutdown()
process2.shutdown()
process3.shutdown()

<h2 style="color:#DE46D3"> Example 2 </h2>
safer usage

In [10]:
def take_0(queue0, queue1, queue2, queue3, event0, event1, event2, event3):
    while not event0.is_set():
        if not queue0.empty():
            message = queue0.get()
            if message == 1:
                event1.clear()
                event2.clear()
                queue1.put(0)
                queue2.put(0)
                event0.set()

def take_1(queue1, event1, event3, queue3):
    while not event1.is_set():
        if not queue1.empty():
            message = queue1.get()
            ran = random.randint(0, 10)
            if ran in [0, 1, 2]:
                print("run found take1")
                event3.clear()
                event1.set()
                time.sleep(2)
                queue3.put("STOP")
            else:
                print("start take 1 execution")
                if message == "STOP":
                    event1.set()

def take_2(queue2, event2, event3, queue3):
    while not event2.is_set():
        if not queue2.empty():
            message = queue2.get()
            ran = random.randint(0, 10)
            if ran in [0, 1, 2]:
                print("run found take2")
                event3.clear()
                queue3.put(3)
                time.sleep(2)
                event2.set()
            else:
                print("start take 2 execution")
                event2.set()

def take_3(queue3, event3, event0):
    while not event3.is_set():
        if not queue3.empty():
            message = queue3.get()
            if message == "STOP":
                event3.set()
                event0.clear()
            else:
                print("take 3:", message)


In [ ]:
queue0 = multiprocessing.Queue()
queue1 = multiprocessing.Queue()
queue2 = multiprocessing.Queue()
queue3 = multiprocessing.Queue()

event0 = multiprocessing.Event()
event1 = multiprocessing.Event()
event2 = multiprocessing.Event()
event3 = multiprocessing.Event()

process0 = multiprocessing.Process(target=take_0, args=(queue0, queue1, queue2, queue3, event0, event1, event2, event3))
process1 = multiprocessing.Process(target=take_1, args=(queue1, event1, event3, queue3))
process2 = multiprocessing.Process(target=take_2, args=(queue2, event2, event3, queue3))
process3 = multiprocessing.Process(target=take_3, args=(queue3, event3, event0))

In [11]:
process0.start()
process1.start()
process2.start()
process3.start()

queue0.put(1)
event0.clear()

# Set the time to run the processes (in seconds)
timeout = 5

def interrupt_processes():
    """ Create a timer thread to interrupt the processes after the timeout. """
    time.sleep(timeout)
    process0.terminate()
    process1.terminate()
    process2.terminate()
    process3.terminate()

timer_thread = threading.Thread(target=interrupt_processes)
timer_thread.start()

#### Wait end of processes, to avoid getting stuck !
process0.join()
process1.join()
process2.join()
process3.join()

#### Don't shutdown!
process0.terminate()
process1.terminate()
process2.terminate()
process3.terminate()

start take 1 executionstart take 2 execution

